In [ ]:
import cv2
cv2.__version__

'4.1.2'

In [ ]:
!nvidia-smi

Sun Jan 10 18:42:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!/usr/local/cuda/bin/nvcc --version
# nvcc -version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


# coba kuy
ini kita nyobain notebook pertama di jupyeterr, semoga lancar yak!!

In [ ]:
!pip install wandb -qqq
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.callbacks import TensorBoard
import random

#Import wandb libraries
import wandb
from wandb.keras import WandbCallback
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
# wandb.init(project="jupyter-projo",
#            config={
#                "batch_size": 128,
#                "learning_rate": 0.01,
#                "dataset": "CIFAR-100",
#            })
# Show graphs in the notebook
# display(wandb.jupyter.Run())
# Initialize wandb and save hyperparameters
wandb.init(
  project="sample-project",
  config={
    "dropout": 0.2,
    "hidden_layer_size": 128,
    "layer_1_size": 16,
    "layer_2_size": 32,
    "learn_rate": 0.01,
    "decay": 1e-6,
    "momentum": 0.9,
    "epochs": 8 }
)
config = wandb.config

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
%%wandb

# Your training loop here
(X_train_orig, y_train_orig), (X_test, y_test) = fashion_mnist.load_data()

# Reducing the dataset size to 10,000 examples for faster train time
true = list(map(lambda x: True if random.random() < 0.167 else False, range(60000)))
ind = []
for i, x in enumerate(true):
    if x == True: ind.append(i)

X_train = X_train_orig[ind, :, :]
y_train = y_train_orig[ind]

img_width=28
img_height=28
labels =["T-shirt/top","Trouser","Pullover","Dress",
    "Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]

X_train = X_train.astype('float32')
X_train /= 255.
X_test = X_test.astype('float32')
X_test /= 255.

#reshape input data
X_train = X_train.reshape(X_train.shape[0], img_width, img_height, 1)
X_test = X_test.reshape(X_test.shape[0], img_width, img_height, 1)

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

sgd = SGD(lr=config.learn_rate, decay=config.decay, momentum=config.momentum,
                            nesterov=True)

# build model
model = Sequential()
model.add(Conv2D(config.layer_1_size, (5, 5), activation='relu',
                            input_shape=(img_width, img_height,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(config.layer_2_size, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(config.dropout))
model.add(Flatten())
model.add(Dense(config.hidden_layer_size, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

#Add Keras WandbCallback
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.fit(X_train, y_train,  validation_data=(X_test, y_test), epochs=config.epochs,
    callbacks=[WandbCallback(data_type="image", labels=labels)])


4423680/4422102 [==============================] - 0s 0us/step
Epoch 1/8
309/309 [==============================] - 9s 28ms/step - loss: 0.9326 - accuracy: 0.6600 - val_loss: 0.6289 - val_accuracy: 0.7675
Epoch 2/8
309/309 [==============================] - 9s 29ms/step - loss: 0.5873 - accuracy: 0.7850 - val_loss: 0.5295 - val_accuracy: 0.8096
Epoch 3/8
309/309 [==============================] - 9s 30ms/step - loss: 0.5185 - accuracy: 0.8121 - val_loss: 0.5128 - val_accuracy: 0.8183
Epoch 4/8
309/309 [==============================] - 9s 29ms/step - loss: 0.4627 - accuracy: 0.8348 - val_loss: 0.4877 - val_accuracy: 0.8286
Epoch 5/8
309/309 [==============================] - 9s 29ms/step - loss: 0.4249 - accuracy: 0.8422 - val_loss: 0.4249 - val_accuracy: 0.8473
Epoch 6/8
  9/309 [..............................] - ETA: 17s - loss: 0.3746 - accuracy: 0.8576